# Evolver Loop 2 Analysis: Overlap Repair Strategy

## Key Insight from Evaluator
The ensemble of best configs scores 67.77 (BELOW target of 68.93!) but has overlaps.
The path to victory is clear: REPAIR THE OVERLAPS.

## Analysis Goals
1. Understand the overlap situation in the best configs
2. Analyze how much improvement is locked behind overlaps
3. Design an overlap repair strategy

In [1]:
import numpy as np
import pandas as pd
import math
from numba import njit
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import os

# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

tx, ty = make_polygon_template()
print('Template loaded')

Template loaded


In [2]:
# Overlap detection functions
def get_shapely_polygon(cx, cy, deg, tx, ty):
    r = deg * np.pi / 180.0
    c = np.cos(r)
    s = np.sin(r)
    px = c * tx - s * ty + cx
    py = s * tx + c * ty + cy
    return Polygon(zip(px, py))

def has_overlap(xs, ys, degs, tx, ty):
    n = len(xs)
    if n <= 1:
        return False
    polygons = [get_shapely_polygon(xs[i], ys[i], degs[i], tx, ty) for i in range(n)]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def find_overlapping_pairs(xs, ys, degs, tx, ty):
    """Find all pairs of overlapping trees"""
    n = len(xs)
    if n <= 1:
        return []
    polygons = [get_shapely_polygon(xs[i], ys[i], degs[i], tx, ty) for i in range(n)]
    tree_index = STRtree(polygons)
    pairs = []
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx <= i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                pairs.append((i, idx))
    return pairs

print('Overlap detection functions loaded')

Overlap detection functions loaded


In [3]:
# Load the ensemble with overlaps (67.77 score)
ensemble_path = '/home/nonroot/snapshots/santa-2025/21108486172/code/experiments/001_baseline/ensemble_submission.csv'
df_ensemble = pd.read_csv(ensemble_path)
df_ensemble['N'] = df_ensemble['id'].astype(str).str.split('_').str[0].astype(int)

# Load baseline (70.73 score, no overlaps)
baseline_path = '/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv'
df_baseline = pd.read_csv(baseline_path)
df_baseline['N'] = df_baseline['id'].astype(str).str.split('_').str[0].astype(int)

print('Loaded ensemble and baseline')

Loaded ensemble and baseline


In [4]:
# Analyze overlaps in the ensemble
overlap_analysis = []

for n in range(1, 201):
    g_ens = df_ensemble[df_ensemble['N'] == n]
    g_base = df_baseline[df_baseline['N'] == n]
    
    xs_ens = strip(g_ens['x'].to_numpy())
    ys_ens = strip(g_ens['y'].to_numpy())
    ds_ens = strip(g_ens['deg'].to_numpy())
    
    xs_base = strip(g_base['x'].to_numpy())
    ys_base = strip(g_base['y'].to_numpy())
    ds_base = strip(g_base['deg'].to_numpy())
    
    score_ens = score_group(xs_ens, ys_ens, ds_ens, tx, ty)
    score_base = score_group(xs_base, ys_base, ds_base, tx, ty)
    
    pairs = find_overlapping_pairs(xs_ens, ys_ens, ds_ens, tx, ty)
    
    overlap_analysis.append({
        'n': n,
        'score_ens': score_ens,
        'score_base': score_base,
        'improvement': score_base - score_ens,
        'num_overlaps': len(pairs),
        'has_overlap': len(pairs) > 0
    })

df_analysis = pd.DataFrame(overlap_analysis)
print(f'Total ensemble score: {df_analysis["score_ens"].sum():.6f}')
print(f'Total baseline score: {df_analysis["score_base"].sum():.6f}')
print(f'Potential improvement: {df_analysis["improvement"].sum():.6f}')
print(f'\nN values with overlaps: {df_analysis["has_overlap"].sum()}')
print(f'N values without overlaps: {(~df_analysis["has_overlap"]).sum()}')

Total ensemble score: 67.772662
Total baseline score: 70.734327
Potential improvement: 2.961665

N values with overlaps: 30
N values without overlaps: 170


In [5]:
# Show N values with most improvement potential (locked behind overlaps)
df_with_overlaps = df_analysis[df_analysis['has_overlap']].sort_values('improvement', ascending=False)
print('Top 20 N values with overlaps and improvement potential:')
print(df_with_overlaps[['n', 'score_ens', 'score_base', 'improvement', 'num_overlaps']].head(20).to_string())

print(f'\nTotal improvement locked behind overlaps: {df_with_overlaps["improvement"].sum():.6f}')

Top 20 N values with overlaps and improvement potential:
     n  score_ens  score_base  improvement  num_overlaps
6    7   0.162010    0.399897     0.237887            21
5    6   0.173625    0.399610     0.225985            15
9   10   0.166604    0.376630     0.210026            41
8    9   0.178013    0.387415     0.209402            34
4    5   0.212694    0.416850     0.204155            10
7    8   0.187564    0.385407     0.197844            27
3    4   0.228621    0.416545     0.187924             6
11  12   0.198679    0.372724     0.174045            48
14  15   0.214172    0.379203     0.165030            61
16  17   0.208784    0.370040     0.161257            55
2    3   0.305312    0.434745     0.129433             3
10  11   0.257940    0.375736     0.117796            30
17  18   0.274445    0.368771     0.094326            36
21  22   0.294317    0.375258     0.080941            58
22  23   0.290307    0.368752     0.078445            48
20  21   0.302114    0.376451  

In [6]:
# Check N values WITHOUT overlaps - these are free improvements!
df_no_overlaps = df_analysis[~df_analysis['has_overlap']]
print(f'N values without overlaps: {len(df_no_overlaps)}')
print(f'Improvement from non-overlapping configs: {df_no_overlaps["improvement"].sum():.6f}')

if len(df_no_overlaps) > 0:
    print('\nThese N values can be used directly from ensemble:')
    for _, row in df_no_overlaps.iterrows():
        if row['improvement'] > 0.0001:
            print(f'  N={row["n"]}: improvement={row["improvement"]:.6f}')

N values without overlaps: 170
Improvement from non-overlapping configs: 0.000000

These N values can be used directly from ensemble:


In [7]:
# Key insight: 170 N values have NO overlaps but also NO improvement
# This means the ensemble only has better configs for N values that HAVE overlaps!
# The 2.96 points of improvement is ALL locked behind overlaps

# Let's understand the overlap structure better
print("Summary of overlap situation:")
print(f"- 30 N values have overlaps (all the improvement is here)")
print(f"- 170 N values have no overlaps (same as baseline)")
print(f"- Total improvement potential: 2.96 points")
print(f"- If we can repair overlaps, we beat the target!")
print(f"\nTarget: 68.931058")
print(f"Baseline: 70.734327")
print(f"Ensemble (with overlaps): 67.772662")
print(f"Gap to target: {68.931058 - 67.772662:.6f} (we're ALREADY below target if overlaps fixed!)")

Summary of overlap situation:
- 30 N values have overlaps (all the improvement is here)
- 170 N values have no overlaps (same as baseline)
- Total improvement potential: 2.96 points
- If we can repair overlaps, we beat the target!

Target: 68.931058
Baseline: 70.734327
Ensemble (with overlaps): 67.772662
Gap to target: 1.158396 (we're ALREADY below target if overlaps fixed!)


In [9]:
# Analyze the overlap structure for a few N values to understand repair difficulty
for test_n in [3, 4, 5, 6, 7]:
    g = df_ensemble[df_ensemble['N'] == test_n]
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    
    pairs = find_overlapping_pairs(xs, ys, ds, tx, ty)
    print(f'\nN={test_n}: {len(pairs)} overlapping pairs out of {test_n} trees')
    improvement = df_analysis[df_analysis["n"]==test_n]["improvement"].values[0]
    print(f'  Improvement potential: {improvement:.6f}')
    
    # Check which trees are involved in overlaps
    involved = set()
    for i, j in pairs:
        involved.add(i)
        involved.add(j)
    print(f'  Trees involved in overlaps: {sorted(involved)}')


N=3: 3 overlapping pairs out of 3 trees
  Improvement potential: 0.129433
  Trees involved in overlaps: [0, 1, 2]

N=4: 6 overlapping pairs out of 4 trees
  Improvement potential: 0.187924
  Trees involved in overlaps: [0, 1, 2, 3]

N=5: 10 overlapping pairs out of 5 trees
  Improvement potential: 0.204155
  Trees involved in overlaps: [0, 1, 2, 3, 4]

N=6: 15 overlapping pairs out of 6 trees
  Improvement potential: 0.225985
  Trees involved in overlaps: [0, 1, 2, 3, 4, 5]

N=7: 21 overlapping pairs out of 7 trees
  Improvement potential: 0.237887
  Trees involved in overlaps: [0, 1, 2, 3, 4, 5, 6]


In [10]:
# Check the cpp_parallel_sa submissions directly to see if any have fewer overlaps
import glob

cpp_sa_files = glob.glob('/home/nonroot/snapshots/santa-2025/21090949260/code/experiments/009_cpp_parallel_sa/submission_cpp*.csv')
print(f'Found {len(cpp_sa_files)} cpp_parallel_sa submissions')

# For each file, count overlaps for N=3,4,5,6,7
for fp in sorted(cpp_sa_files)[:5]:  # Check first 5
    df = pd.read_csv(fp)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    overlap_counts = []
    for test_n in [3, 4, 5, 6, 7]:
        g = df[df['N'] == test_n]
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        pairs = find_overlapping_pairs(xs, ys, ds, tx, ty)
        overlap_counts.append(len(pairs))
    
    print(f'{fp.split("/")[-1]}: overlaps for N=3,4,5,6,7 = {overlap_counts}')

Found 20 cpp_parallel_sa submissions
submission_cpp.csv: overlaps for N=3,4,5,6,7 = [0, 0, 0, 0, 7]
submission_cpp10.csv: overlaps for N=3,4,5,6,7 = [3, 0, 3, 0, 7]
submission_cpp11.csv: overlaps for N=3,4,5,6,7 = [3, 1, 6, 1, 7]
submission_cpp12.csv: overlaps for N=3,4,5,6,7 = [3, 5, 6, 1, 11]
submission_cpp13.csv: overlaps for N=3,4,5,6,7 = [3, 6, 6, 1, 11]


In [11]:
# Check submission_cpp.csv more thoroughly - it seems to have fewer overlaps!
cpp_path = '/home/nonroot/snapshots/santa-2025/21090949260/code/experiments/009_cpp_parallel_sa/submission_cpp.csv'
df_cpp = pd.read_csv(cpp_path)
df_cpp['N'] = df_cpp['id'].astype(str).str.split('_').str[0].astype(int)

# Calculate total score and count overlaps for all N
cpp_scores = {}
cpp_overlaps = {}
for n in range(1, 201):
    g = df_cpp[df_cpp['N'] == n]
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    cpp_scores[n] = score_group(xs, ys, ds, tx, ty)
    pairs = find_overlapping_pairs(xs, ys, ds, tx, ty)
    cpp_overlaps[n] = len(pairs)

total_cpp = sum(cpp_scores.values())
n_with_overlaps = sum(1 for n, v in cpp_overlaps.items() if v > 0)
print(f'submission_cpp.csv total score: {total_cpp:.6f}')
print(f'N values with overlaps: {n_with_overlaps}')
print(f'N values without overlaps: {200 - n_with_overlaps}')

submission_cpp.csv total score: 84.596364
N values with overlaps: 183
N values without overlaps: 17


In [12]:
# Strategy: Create a new ensemble by picking best VALID (no overlap) config for each N
# from ALL available submission files

import glob

all_files = glob.glob('/home/nonroot/snapshots/**/*.csv', recursive=True)
print(f'Found {len(all_files)} CSV files total')

# Filter for submission-like files
submission_files = [f for f in all_files if 'submission' in f.lower() or 'santa' in f.lower()]
print(f'Found {len(submission_files)} potential submission files')

Found 236 CSV files total
Found 236 potential submission files


In [13]:
# Build best VALID ensemble from all sources
from tqdm import tqdm

best_valid = {n: {'score': 1e300, 'data': None, 'src': None} for n in range(1, 201)}

for fp in tqdm(submission_files, desc='Scanning'):
    try:
        df = pd.read_csv(fp)
    except Exception:
        continue
    
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        continue
    
    # Check if values have 's' prefix
    sample_x = str(df['x'].iloc[0])
    if not sample_x.startswith('s'):
        continue
    
    df = df.copy()
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    for n, g in df.groupby('N'):
        if n < 1 or n > 200:
            continue
        
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        
        # Check for overlaps - only keep valid configs
        if has_overlap(xs, ys, ds, tx, ty):
            continue
        
        sc = score_group(xs, ys, ds, tx, ty)
        if sc < best_valid[n]['score']:
            best_valid[n]['score'] = float(sc)
            best_valid[n]['data'] = g.drop(columns=['N']).copy()
            best_valid[n]['src'] = fp

print('\\nBest valid ensemble built!')

Scanning:   0%|          | 0/236 [00:00<?, ?it/s]

Scanning:   0%|          | 1/236 [00:01<06:31,  1.66s/it]

Scanning:   1%|          | 2/236 [00:03<06:15,  1.61s/it]

Scanning:   1%|▏         | 3/236 [00:04<06:09,  1.59s/it]

Scanning:   2%|▏         | 4/236 [00:06<05:48,  1.50s/it]

...

Scanning:  99%|█████████▉| 234/236 [05:02<00:02,  1.14s/it]

Scanning: 100%|█████████▉| 235/236 [05:03<00:01,  1.00s/it]

Scanning: 100%|██████████| 236/236 [05:04<00:00,  1.16s/it]

Scanning: 100%|██████████| 236/236 [05:04<00:00,  1.29s/it]

\nBest valid ensemble built!


In [14]:
# Calculate the best valid ensemble score
valid_ensemble_score = sum(best_valid[n]['score'] for n in range(1, 201))
print(f'Best valid ensemble score: {valid_ensemble_score:.6f}')
print(f'Baseline score: 70.734327')
print(f'Target: 68.931058')
print(f'Gap to target: {valid_ensemble_score - 68.931058:.6f}')

# Check how many N values improved
improved_n = []
for n in range(1, 201):
    baseline_score = df_analysis[df_analysis['n']==n]['score_base'].values[0]
    if best_valid[n]['score'] < baseline_score - 0.0001:
        improved_n.append(n)

print(f'\\nN values with improvement: {len(improved_n)}')

Best valid ensemble score: 70.734327
Baseline score: 70.734327
Target: 68.931058
Gap to target: 1.803269
\nN values with improvement: 0


In [ ]:
# Analyze a specific N value with overlaps to understand the repair challenge
test_n = 10  # Pick a small N to analyze

g = df_ensemble[df_ensemble['N'] == test_n]
xs = strip(g['x'].to_numpy())
ys = strip(g['y'].to_numpy())
ds = strip(g['deg'].to_numpy())

pairs = find_overlapping_pairs(xs, ys, ds, tx, ty)
print(f'N={test_n}: {len(pairs)} overlapping pairs')
for i, j in pairs:
    print(f'  Trees {i} and {j} overlap')
    print(f'    Tree {i}: x={xs[i]:.4f}, y={ys[i]:.4f}, deg={ds[i]:.2f}')
    print(f'    Tree {j}: x={xs[j]:.4f}, y={ys[j]:.4f}, deg={ds[j]:.2f}')